In [24]:
from langchain.schema import Document
from langchain_community.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")



docs = [
    Document(
        page_content="""
        At Bresto Café, we specialize in artisan coffee blends carefully crafted from the finest beans 
        around the world. Our signature brews range from bold dark roasts to delicate single-origin 
        pour-overs. We believe in sustainable sourcing and work directly with small farms to ensure 
        quality and ethics. Coffee lovers can also enjoy our rotating seasonal specials and in-house 
        cold brews, made fresh daily.
        """,
        metadata={"source": "CoffeeMenu.txt"}
    ),

    Document(
        page_content="""
        The brunch experience at Bresto is a local favorite. From fluffy sourdough pancakes topped with 
        wild berries to savory avocado toast served on freshly baked multigrain bread, every dish is made 
        with locally sourced ingredients. Our vegan and gluten-free options make Bresto an inclusive spot 
        for all dietary needs. Don't miss our house-special: shakshuka with sour cream and za'atar toast.
        """,
        metadata={"source": "BrunchMenu.txt"}
    ),

    Document(
        page_content="""
        At Bresto, we care about the environment. All our packaging is compostable, and we’ve eliminated 
        single-use plastics. We also host community sustainability events, such as zero-waste workshops 
        and local farm pop-ups. Our rooftop herb garden provides ingredients for our teas and specials, 
        helping reduce food miles and promote urban greenery.
        """,
        metadata={"source": "SustainabilityPolicy.txt"}
    )
]

db =  Chroma.from_documents(docs,embedding_model)

In [25]:
retriever = db.as_retriever(search_kwargs={"k":3})

In [26]:
retriever.invoke("what is Bresto Café")

[Document(metadata={'source': 'CoffeeMenu.txt'}, page_content='\n        At Bresto Café, we specialize in artisan coffee blends carefully crafted from the finest beans \n        around the world. Our signature brews range from bold dark roasts to delicate single-origin \n        pour-overs. We believe in sustainable sourcing and work directly with small farms to ensure \n        quality and ethics. Coffee lovers can also enjoy our rotating seasonal specials and in-house \n        cold brews, made fresh daily.\n        '),
 Document(metadata={'source': 'CoffeeMenu.txt'}, page_content='\n        At Bresto Café, we specialize in artisan coffee blends carefully crafted from the finest beans \n        around the world. Our signature brews range from bold dark roasts to delicate single-origin \n        pour-overs. We believe in sustainable sourcing and work directly with small farms to ensure \n        quality and ethics. Coffee lovers can also enjoy our rotating seasonal specials and in-hou

In [27]:
from langchain_core.prompts import ChatPromptTemplate
template = """
    Answer the question based on context :{context}
    Question {question}
    """
prompt = ChatPromptTemplate.from_template(template)

In [28]:
from langchain_core.output_parsers import StrOutputParser
from langchain_groq import ChatGroq
llm = ChatGroq(model="llama-3.1-8b-instant")

def formatDoc(docs):
    return "\n\n".join(f"{doc.page_content}\nSource: {doc.metadata['source']}" for doc in docs)


chain = (
    {
        "context":lambda x:formatDoc(retriever.invoke(x)),
        "question":lambda x:x
    }
    |prompt
    |llm
    |StrOutputParser()

)    

chain.invoke("what is Bresto cafe")

'Based on the context provided in the text, Bresto Café appears to be a coffee shop or café that specializes in artisan coffee blends.'